<a href="https://colab.research.google.com/github/gulizsamgar/PYTHON-CLTV-Prediction-with-BG-NBD-and-Gamma-Gamma/blob/main/FLO_CLTV_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BG-NBD ve Gamma-Gamma ile CLTV Tahmini**

* * *

*Proje Yazarı: Güliz Samğar*

*Son Güncelleme Tarihi: 19 Ağustos 2025*

* * *

**Proje Genel Bakış**

FLO, satış ve pazarlama faaliyetleri için stratejik bir yol haritası belirlemek istemektedir. Şirketin orta ve uzun vadeli planlarını yapabilmesi amacıyla, mevcut müşterilerin gelecekte şirkete sağlayacakları potansiyel değerin (Müşteri Yaşam Boyu Değeri - CLTV) tahmin edilmesi gerekmektedir. Bu proje kapsamında, FLO’nun 20.000 müşteriden oluşan veri seti üzerinde bu tahminler gerçekleştirilecektir.

Çalışmanın amacı, müşterilerin geçmiş alışveriş davranışlarını yansıtan RFM metriklerini (Recency, Frequency, Monetary) kullanarak gelecekteki değerlerini (CLTV) tahmin etmek, bu tahmine dayanarak müşteri segmentasyonu yapmak ve hedef pazarlama stratejileri oluşturmaktır. FLO, tahmini CLTV değerlerine göre müşterilerini segmentlere ayırıp bu segmentlere özel pazarlama stratejileri belirlemek istiyor. Buna yönelik olarak, müşterilerin geçmiş davranışlarından elde edilen RFM metrikleri CLTV modelleri için girdi olarak kullanılacak ve bu modellerle hesaplanan CLTV değerlerine göre gruplar oluşturulacaktır.

Proje Adımları:

1.  **Veriyi Tanıma ve Hazırlama:** Veri setinin yüklenmesi, genel bilgilerin incelenmesi, eksik değer kontrolü ve yeni değişkenlerin (toplam sipariş, toplam harcama) oluşturulması.
2.  **CLTV Metriklerinin Hesaplanması:** Recency (İlk ve son sipariş arasındaki süre), T (İlk sipariş ve analiz tarihi arasındaki süre), Frequency (Toplam sipariş sayısı) ve Monetary (Ortalama işlem değeri) gibi CLTV metriklerinin hesaplanması.
3.  **BG/NBD Modeli ile Gelecek Satın Alma Sayısı Tahmini:** Müşterilerin gelecekte belirli bir zaman diliminde yapması beklenen satın alma sayısının BG/NBD modeli kullanılarak tahmin edilmesi.
4.  **Gamma-Gamma Modeli ile Ortalama İşlem Değeri Tahmini:** Müşterilerin gelecekteki her bir işleminde harcaması beklenen ortalama değerin Gamma-Gamma modeli kullanılarak tahmin edilmesi.
5.  **CLTV Hesaplama:** BG/NBD ve Gamma-Gamma modellerinin çıktıları birleştirilerek her müşterinin belirli bir zaman dilimi (örneğin 6 ay) için Müşteri Yaşam Boyu Değeri'nin (CLTV) hesaplanması.
6.  **CLTV'ye Göre Segmentlerin Oluşturulması:** Hesaplanan CLTV değerlerine dayanarak müşterilerin farklı segmentlere ayrılması (örneğin A, B, C, D gibi).
7.  **Eyleme Dönüştürülebilir İçgörüler:** Oluşturulan CLTV segmentlerinin özelliklerinin incelenmesi ve her bir segmente yönelik özelleştirilmiş pazarlama ve müşteri ilişkileri yönetimi stratejilerinin belirlenmesi.
8.  **BONUS: Tüm Süreci Fonksiyonlaştırma:** CLTV hesaplama ve segmentasyon adımlarının tekrar kullanılabilir bir fonksiyon içine alınması.

Bu analiz sonucunda elde edilen müşteri segmentleri ve CLTV tahminleri, FLO'nun pazarlama faaliyetlerini daha etkili ve kişiselleştirilmiş hale getirmesine, kaynakları daha verimli kullanmasına ve orta/uzun vadeli stratejik planlamasına yardımcı olacaktır.

Bu projede kullanılan veri setine [Github'dan](https://github.com/gulizsamgar/PYTHON-CLTV-Prediction-with-BG-NBD-and-Gamma-Gamma/blob/4498a795dfa20ef22fd5c717161cde886ee6bc5e/dataset/flo_data_20k.csv) ulaşabilirsiniz.

> *Veri setimizle çalışmaya hazırız. Haydi başlayalım!*



***

**Kütüphanelerin İçe Aktarılması ve Görüntü Ayarları**

Bu bölüm, veri analizi için gerekli kütüphaneleri içe aktarır ve pandas görüntü ayarlarını yapılandırır.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# lifetimes kütüphanesini yükle
!pip install lifetimes

In [ ]:
# Gerekli kütüphaneleri import et

import pandas as pd
import datetime as dt
from lifetimes import BetaGeoFitter, GammaGammaFitter
from sklearn.preprocessing import MinMaxScaler

# Pandas gösterim ayarlarını yapılandır
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.mode.chained_assignment = None

Yukarıdaki kod bloğu, veri analizi ve CLTV (Customer Lifetime Value) tahmini için gerekli olan Python kütüphanelerini çalışma ortamına dahil eder ve veri görselleştirmesini kolaylaştırmak amacıyla pandas kütüphanesinin görüntüleme seçeneklerini ayarlar.

**İçe Aktarılan Kütüphaneler:**

*   `import pandas as pd`: Veri manipülasyonu ve analizi için güçlü bir kütüphane olan pandas'ı içe aktarır ve yaygın olarak kullanılan `pd` alias'ı ile kullanılmasını sağlar.
*   `import datetime as dt`: Tarih ve saat verileriyle çalışmak için gerekli fonksiyonları sağlar. Bu projede müşteri sipariş tarihlerini işlemek için kullanılacaktır.
*   `from lifetimes import BetaGeoFitter, GammaGammaFitter`: CLTV tahmini için BG-NBD (Beta-Geometric Negative Binomial Distribution) ve Gamma-Gamma modellerini içeren `lifetimes` kütüphanesinden ilgili sınıfları içe aktarır. Bu modeller, müşteri yaşam boyu değerini tahmin etmek için kullanılır.
*   `from sklearn.preprocessing import MinMaxScaler`: scikit-learn kütüphanesinden `MinMaxScaler` sınıfını içe aktarır. Bu sınıf, veri setindeki sayısal sütunları belirli bir aralığa (genellikle 0 ile 1 arasına) ölçeklendirmek için kullanılır. Bu, bazı makine öğrenimi algoritmaları veya modelleme adımları için faydalı olabilir.

**Pandas Görüntü Ayarları:**

*   `pd.set_option('display.max_columns', None)`: Pandas DataFrame'leri yazdırılırken sütun sayısının sınırlanmamasını sağlar. Bu sayede, tüm sütunlar görüntülenebilir ve veri setindeki tüm değişkenler tek bir çıktıda incelenebilir.
*   `pd.set_option('display.max_rows', None)`: Pandas DataFrame'leri yazdırılırken satır sayısının sınırlanmamasını sağlar. Bu, büyük veri setlerinde bile tüm gözlemlerin görüntülenmesine olanak tanır.
*   `pd.set_option('display.float_format', lambda x: '%.2f' % x)`: Pandas DataFrame'lerindeki ondalık sayıların (float) virgülden sonra yalnızca iki basamak kullanılarak biçimlendirilmesini sağlar. Bu, özellikle parasal değerler gibi ondalık verilerin daha düzenli ve okunabilir görünmesini sağlar.
*   `pd.options.mode.chained_assignment = None`: Pandas'ta zincirleme atama (chained assignment) ile ilgili potansiyel uyarıları kapatır. Zincirleme atama, DataFrame'in bir görünümüne (view) veya kopyasına (copy) veri ataması yapmaya çalışıldığında ortaya çıkabilir ve beklenen sonucu vermeyebilir. Bu ayarın `None` olarak ayarlanması, bu tür durumlarda pandas'ın uyarı vermesini engeller. Ancak bu ayar kullanılırken dikkatli olunmalıdır, çünkü kapatılan uyarılar önemli potansiyel hataları gösterebilir.

Bu ayarlar ve kütüphanelerin içe aktarılması, projenin ilerleyen adımlarında veri yükleme, temizleme, dönüştürme ve modelleme işlemlerini gerçekleştirmek için gerekli temel altyapıyı hazırlar.

---

**GÖREV 1: VERİYİ TANIMA VE HAZIRLAMA**

In [ ]:
###############################################################
# 1. Veri setini içe aktar
###############################################################

# Dosya yolu tanımlanıyor.
file_path = '/content/drive/MyDrive/flo_data_20k.csv'
# CSV dosyasını pandas DataFrame'ine oku.
df_ = pd.read_csv(file_path)
# Orijinal DataFrame'in bir kopyasını oluştur.
df = df_.copy()

# Doğru şekilde yüklendiğini onaylamak için ilk birkaç satırı görüntüle
print("\nFirst 5 rows of dataset:")
print(df.head(), "\n")


First 5 rows of dataset:
                              master_id order_channel last_order_channel  \
0  cc294636-19f0-11eb-8d74-000d3a38a36f   Android App            Offline   
1  f431bd5a-ab7b-11e9-a2fc-000d3a38a36f   Android App             Mobile   
2  69b69676-1a40-11ea-941b-000d3a38a36f   Android App        Android App   
3  1854e56c-491f-11eb-806e-000d3a38a36f   Android App        Android App   
4  d6ea1074-f1f5-11e9-9346-000d3a38a36f       Desktop            Desktop   

  first_order_date last_order_date last_order_date_online  \
0       2020-10-30      2021-02-26             2021-02-21   
1       2017-02-08      2021-02-16             2021-02-16   
2       2019-11-27      2020-11-27             2020-11-27   
3       2021-01-06      2021-01-17             2021-01-17   
4       2019-08-03      2021-03-07             2021-03-07   

  last_order_date_offline  order_num_total_ever_online  \
0              2021-02-26                         4.00   
1              2020-01-10         

  **Veri Seti Bilgisi**

  Bu veri seti, 2020 - 2021 yıllarında OmniChannel (hem online hem offline alışveriş yapan) olarak alışveriş yapan müşterilerin geçmiş alışveriş davranışlarından elde edilen bilgilerden oluşmaktadır.

  Veri setindeki sütunlar aşağıdaki bilgileri içermektedir:

*   **master\_id**: Eşsiz müşteri numarası
*   **order\_channel**: Alışveriş yapılan platforma ait hangi kanalın kullanıldığı (Android, ios, Desktop, Mobile, Offline)
*   **last\_order\_channel**: En son alışverişin yapıldığı kanal
*   **first\_order\_date**: Müşterinin yaptığı ilk alışveriş tarihi
*   **last\_order\_date**: Müşterinin yaptığı son alışveriş tarihi
*   **last\_order\_date\_online**: Müşterinin online platformda yaptığı son alışveriş tarihi
*   **last\_order\_date\_offline**: Müşterinin offline platformda yaptığı son alışveriş tarihi
*   **order\_num\_total\_ever\_online**: Müşterinin online platformda yaptığı toplam alışveriş sayısı
*   **order\_num\_total\_ever\_offline**: Müşterinin offline'da yaptığı toplam alışveriş sayısı
*   **customer\_value\_total\_ever\_offline**: Müşterinin offline alışverişlerinde ödediği toplam ücret
*   **customer\_value\_total\_ever\_online**: Müşterinin online alışverişlerinde ödediği toplam ücret
*   **interested\_in\_categories\_12**: Müşterinin son 12 ayda alışveriş yaptığı kategorilerin listesi

---

In [ ]:
###############################################################
# 2. Aykırı değerleri baskılamak için fonksiyonlar
###############################################################

# Aykırı değerler için eşik değerlerini hesaplayan fonksiyon
def outlier_thresholds(dataframe, variable):
    # %1 ve %99 çeyreklerini (quartiles) hesapla
    q1 = dataframe[variable].quantile(0.01)
    q3 = dataframe[variable].quantile(0.99)
    # Çeyrekler arası açıklığı (Interquartile Range - IQR) hesapla
    iqr = q3 - q1
    # Üst eşik değerini hesapla
    up_limit = q3 + 1.5 * iqr
    # Alt eşik değerini hesapla
    low_limit = q1 - 1.5 * iqr
    # Hesaplanan eşik değerlerini döndür (yuvarlanmış)
    return round(low_limit, 0), round(up_limit, 0)

# Aykırı değerleri eşik değerleri ile değiştiren fonksiyon
def replace_with_thresholds(dataframe, variable):
    # Belirtilen değişken için alt ve üst eşik değerlerini al
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    # Alt eşiğin altındaki değerleri alt eşik ile değiştir
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    # Üst eşiğin üzerindeki değerleri üst eşik ile değiştir
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

* * *

**Aykırı Değerleri Baskılamak İçin Fonksiyon Tanımlama**

Bu adımda, veri setimizdeki olası aykırı değerleri (outliers) tespit etmek ve analizimizin doğruluğunu artırmak amacıyla bu değerleri belirli eşiklerle değiştirmek için iki adet yardımcı Python fonksiyonu tanımlanmıştır. Bu fonksiyonlar, özellikle CLTV hesaplamaları gibi hassas analizler öncesinde veri setini temizlemek için kritik öneme sahiptir.

Tanımlanan fonksiyonlar şunlardır:

1.  **`outlier_thresholds(dataframe, variable)` Fonksiyonu:**
    *   Bu fonksiyonun amacı, verilen bir DataFrame'deki belirli bir sayısal değişken (sütun) için aykırı değerlerin kabul edilebilir alt ve üst sınırlarını hesaplamaktır.
    *   Bu sınırlar belirlenirken, değişkenin değer dağılımının %1 ve %99'luk çeyrekleri (quartiles) kullanılır. Bu çeyrekler arasındaki fark (Çeyrekler Arası Açıklık - IQR) hesaplanır.
    *   Alt eşik, %1'lik çeyrek değerinden 1.5 * IQR çıkarılarak; üst eşik ise %99'luk çeyrek değerine 1.5 * IQR eklenerek belirlenir. Bu yöntem, yaygın olarak kullanılan ve dağılımın dışındaki aşırı uç değerleri belirlemede etkili bir yöntemdir.
    *   **Önemli Not:** CLTV hesaplamaları için frekans (alışveriş sayısı) değerlerinin tam sayı (integer) olması gerektiğinden, hesaplanan alt ve üst eşik değerleri `round()` fonksiyonu ile en yakın tam sayıya yuvarlanır.
    *   Fonksiyon, hesapladığı bu alt ve üst eşik değerlerini döndürür.

2.  **`replace_with_thresholds(dataframe, variable)` Fonksiyonu:**
    *   Bu fonksiyon, `outlier_thresholds` fonksiyonu tarafından hesaplanan alt ve üst eşik değerlerini kullanarak veri setindeki aykırı değerleri düzeltir.
    *   Fonksiyon, belirtilen değişkendeki her bir değeri kontrol eder.
    *   Eğer bir değer, hesaplanan alt eşiğin altındaysa, o değer alt eşik değeri ile değiştirilir.
    *   Eğer bir değer, hesaplanan üst eşiğin üzerindeyse, o değer üst eşik değeri ile değiştirilir.
    *   Bu işlem, veri setindeki aşırı uç değerlerin neden olabileceği çarpıklıkları veya yanlış sonuçları önler. Veri setini daha standart ve güvenilir hale getirerek sonraki analiz ve modelleme adımlarının doğruluğunu artırır.

Bu fonksiyonlar bir bütün olarak, veri setindeki sayısal değişkenlerin analiz ve modelleme için daha uygun hale getirilmesine yardımcı olur.

---

In [ ]:
###############################################################
# 3. İlgili değişkenlerde aykırı değer baskılama
###############################################################

# Aykırı değerleri baskılanacak sütunlar
columns = ["order_num_total_ever_online", "order_num_total_ever_offline",
           "customer_value_total_ever_offline", "customer_value_total_ever_online"]

# Belirlenen sütunlarda aykırı değerleri baskıla
for col in columns:
    replace_with_thresholds(df, col)

# İşlem tamamlandı mesajı
print("Outliers handled successfully!")

Outliers handled successfully!


* * *

**Aykırı Değer Baskılama İşleminin Tamamlanması**

Bu adımda, daha önce tanımladığımız `replace_with_thresholds` fonksiyonu kullanılarak, veri setindeki toplam online ve offline sipariş sayıları (`order_num_total_ever_online`, `order_num_total_ever_offline`) ile toplam online ve offline harcama değerleri (`customer_value_total_ever_online`, `customer_value_total_ever_offline`) değişkenlerindeki aykırı değerler belirlenen eşik değerleri ile değiştirilmiştir.

Bu işlem, veri setini temizleyerek ve aşırı uç değerlerin potansiyel etkisini azaltarak sonraki analiz ve modelleme adımları için daha güvenilir bir temel oluşturmayı amaçlar.

---

In [ ]:
###############################################################
# 4. Özellik Mühendisliği: Müşteri bazında toplam sipariş sayısı ve toplam harcama
###############################################################

# Online ve offline sipariş sayılarını toplayarak toplam sipariş sayısını hesapla
df["order_num_total"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]

# Online ve offline harcama değerlerini toplayarak toplam harcama değerini hesapla
df["customer_value_total"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

# Yeni oluşturulan değişkenleri kontrol etmek için DataFrame'in ilk birkaç satırını görüntüle (isteğe bağlı)
display(df.head())

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,order_num_total,customer_value_total
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.00,1.00,139.99,799.38,[KADIN],5.00,939.37
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.00,2.00,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.00,2013.55
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.00,2.00,189.97,395.35,"[ERKEK, KADIN]",5.00,585.32
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.00,1.00,39.99,81.98,"[AKTIFCOCUK, COCUK]",2.00,121.97
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.00,1.00,49.99,159.99,[AKTIFSPOR],2.00,209.98


* * *

**Toplam Alışveriş Sayısı ve Harcama Değişkenlerinin Oluşturulması**

Bu adımda, her bir müşterinin online ve offline platformlardaki alışveriş verileri birleştirilerek iki yeni değişken oluşturulmuştur:

*   **`order_num_total`**: Müşterinin online (`order_num_total_ever_online`) ve offline (`order_num_total_ever_offline`) kanallardan yaptığı toplam sipariş sayısını temsil eder. Bu değişken, müşterinin genel alışveriş sıklığını gösterir.
*   **`customer_value_total`**: Müşterinin online (`customer_value_total_ever_online`) ve offline (`customer_value_total_ever_offline`) kanallarda harcadığı toplam tutarı temsil eder. Bu değişken, müşterinin toplam değerini ve harcama potansiyelini yansıtır.

Bu yeni değişkenler, müşterilerin genel alışveriş davranışlarını daha iyi anlamak ve RFM veya CLTV gibi ileri analizler için temel sağlamak amacıyla oluşturulmuştur.

---

**GÖREV 2: CLTV METRİKLERİNİN HESAPLANMASI**

Bu bölüm, Customer Lifetime Value (CLTV) tahmini için temel oluşturan Recency, Frequency ve Monetary gibi müşteri metriklerini hesaplar.

In [ ]:
###############################################################
# 1. Analiz tarihi: Son alışveriş tarihinden 2 gün sonrası
###############################################################

# 'last_order_date' ve 'first_order_date' sütunlarını datetime formatına çevir
for col in ["last_order_date", "first_order_date"]:
    df[col] = pd.to_datetime(df[col])

# Veri setindeki son sipariş tarihini bul
print("\nLast order date in dataset:")
print(df["last_order_date"].max(),"\n")

# Analiz tarihini belirle (son sipariş tarihinden 2 gün sonrası)
analysis_date = dt.datetime(2021, 6, 1)

###############################################################
# 2. CLTV dataframe oluşturma
###############################################################

# CLTV analizi için yeni DataFrame oluştur
cltv_df = pd.DataFrame()
# Müşteri ID'lerini ekle
cltv_df["customer_id"] = df["master_id"]
# Recency_cltv_weekly hesapla (haftalık)
cltv_df["recency_cltv_weekly"] = ((df["last_order_date"] - df["first_order_date"]).dt.days) / 7
# T_weekly hesapla (haftalık)
cltv_df["T_weekly"] = ((analysis_date - df["first_order_date"]).dt.days) / 7
# Frequency ata (toplam sipariş sayısı)
cltv_df["frequency"] = df["order_num_total"]
# Monetary_cltv_avg hesapla (ortalama işlem değeri)
cltv_df["monetary_cltv_avg"] = df["customer_value_total"] / df["order_num_total"]

# Oluşturulan CLTV DataFrame'inin ilk birkaç satırını görüntüle
print("\nCLTV dataframe created!")
print(cltv_df.head(),"\n")


Last order date in dataset:
2021-05-30 00:00:00 


CLTV dataframe created!
                            customer_id  recency_cltv_weekly  T_weekly  \
0  cc294636-19f0-11eb-8d74-000d3a38a36f                17.00     30.57   
1  f431bd5a-ab7b-11e9-a2fc-000d3a38a36f               209.86    224.86   
2  69b69676-1a40-11ea-941b-000d3a38a36f                52.29     78.86   
3  1854e56c-491f-11eb-806e-000d3a38a36f                 1.57     20.86   
4  d6ea1074-f1f5-11e9-9346-000d3a38a36f                83.14     95.43   

   frequency  monetary_cltv_avg  
0       5.00             187.87  
1      21.00              95.88  
2       5.00             117.06  
3       2.00              60.98  
4       2.00             104.99   



* * *

**CLTV Metriklerinin Hesaplama Adımı**

Bu adımda, Customer Lifetime Value (CLTV) analizi için temel teşkil eden metrikler hesaplanarak yeni bir DataFrame (`cltv_df`) oluşturulmuştur.

1.  **Analiz Tarihinin Belirlenmesi:**
    *   Veri setindeki en güncel müşteri aktivitesini yansıtması için, analiz tarihi veri setindeki en son sipariş tarihinden 2 gün sonrası olarak belirlenmiştir. Bu tarih (`analysis_date`), `dt.datetime(2021, 6, 1)` olarak sabitlenmiştir.

2.  **CLTV DataFrame'inin Oluşturulması (`cltv_df`):**
    *   Müşteri bazında CLTV metriklerini tutacak boş bir DataFrame oluşturulur.
    *   Her müşterinin benzersiz kimliği (`master_id`), `customer_id` sütunu olarak DataFrame'e eklenir.
    *   **`recency_cltv_weekly`**: Müşterinin ilk siparişi ile son siparişi arasındaki süreyi hafta cinsinden hesaplar. Bu değer, müşterinin aktif olduğu süreyi gösterir. Formül: `(Son Sipariş Tarihi - İlk Sipariş Tarihi) / 7 gün`.
    *   **`T_weekly`**: Müşterinin ilk sipariş tarihi ile belirlenen analiz tarihi arasındaki toplam süreyi hafta cinsinden hesaplar. Bu, müşterinin sistemde ne kadar süredir var olduğunu gösterir. Formül: `(Analiz Tarihi - İlk Sipariş Tarihi) / 7 gün`.
    *   **`frequency`**: Müşterinin veri setindeki toplam sipariş sayısını alır. Bu değer daha önceki adımlarda online ve offline siparişlerin toplamı olarak hesaplanmıştı (`order_num_total`).
    *   **`monetary_cltv_avg`**: Her müşterinin ortalama işlem değerini hesaplar. Bu, toplam harcama değerinin toplam sipariş sayısına bölünmesiyle elde edilir. Formül: `Toplam Harcama / Toplam Sipariş Sayısı`. Dikkat edilmesi gereken nokta, toplam sipariş sayısı 0 olan müşteriler için bu değer tanımsız (NaN) olacaktır.

Bu adımlar tamamlandığında, `cltv_df` DataFrame'i BG-NBD ve Gamma-Gamma modelleri ile CLTV tahmini yapmak için gerekli temel müşteri bazlı metrikleri içermiş olur.

---

---

**GÖREV 3: BG/NBD ve GAMMA-GAMMA MODELLERİ İLE CLTV HESAPLAMA**

Bu bölüm, müşteri yaşam boyu değerini (CLTV) tahmin etmek için BG/NBD ve Gamma-Gamma istatistiksel modellerini kullanarak hesaplamalar yapar.

In [ ]:
###############################################################
# 1. BG/NBD Modeli
###############################################################

# BG/NBD modelini uydurmadan önce veri hazırlığı
# Sıklığı 0 olan müşterileri çıkar
cltv_df = cltv_df[cltv_df['frequency'] > 0]

# Frequency sütununu integer'a dönüştür
cltv_df['frequency'] = cltv_df['frequency'].astype(int)

# Model uydurmadan önce frequency ve recency_cltv_weekly minimum değerlerini kontrol et
print(f"\nMinimum frequency after filtering: {cltv_df['frequency'].min()}")
print(f"Minimum recency_cltv_weekly after filtering: {cltv_df['recency_cltv_weekly'].min()}")

# Beta-Geometric Negative Binomial Distribution (BG/NBD) modelini başlat
# penalizer_coef, aşırı uyumu (overfitting) önlemek için kullanılan bir düzenlileştirme (regularization) parametresidir.
bgf = BetaGeoFitter(penalizer_coef=0.001)

# BG/NBD modelini veri setine uygula (fit et)
# Model, müşterilerin satın alma sıklığı (frequency), ilk ve son satın alma arasındaki süre (recency_cltv_weekly)
# ve ilk satın alma ile analiz tarihi arasındaki süre (T_weekly) verilerini kullanarak parametrelerini öğrenir.
bgf.fit(cltv_df['frequency'],
        cltv_df['recency_cltv_weekly'],
        cltv_df['T_weekly'])

# Modelin başarıyla kurulduğunu belirten bir mesaj yazdır
print("BG/NBD model fitted successfully!")


Minimum frequency after filtering: 2
Minimum recency_cltv_weekly after filtering: 0.0
BG/NBD model fitted successfully!


/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


* * *

**BG/NBD Modelinin Kurulması**

Bu adımda, müşteri yaşam boyu değerini (CLTV) tahmin etmek için kullanılan iki ana modelden ilki olan BG/NBD (Beta-Geometric Negative Binomial Distribution) modeli kurulmuştur.

*   **BG/NBD Modeli Ne Yapar?**
    *   Bu model, bir müşterinin gelecekteki belirli bir zaman diliminde yapacağı tahmini satın alma sayısını belirler.
    *   Müşterinin ne zaman "churn" (terk etme) olasılığının daha yüksek olduğunu ve ne zaman hala "alive" (aktif) olduğunu olasılıksal olarak modeller.
    *   Model, müşterinin geçmiş satın alma sıklığı (frequency), ilk ve son satın alma arasındaki süre (recency) ve ilk satın alma ile analiz tarihi arasındaki toplam süre (T) gibi bilgileri kullanarak öğrenir.

*   **Kod Açıklaması:**

    1.  **Veri Hazırlığı:** BG/NBD modelini uydurmadan önce, `cltv_df` DataFrame'i hazırlanır. Bu hazırlık kapsamında sıklığı (frequency) 0 olan müşteriler çıkarılır ve `frequency` sütunu integer tipine dönüştürülür. Bu adımlar, modelin doğru çalışması ve hata vermemesi için gereklidir.

    2.  **Modelin Başlatılması:** `BetaGeoFitter(penalizer_coef=0.001)` kodu ile BG/NBD modeli başlatılır. `penalizer_coef` parametresi, modelin veri setine aşırı uyumunu (overfitting) önlemek için kullanılan küçük bir düzenlileştirme (regularization) değeridir.

    3.  **Modelin Eğitilmesi (Fit Edilmesi):** `bgf.fit(cltv_df['frequency'], cltv_df['recency_cltv_weekly'], cltv_df['T_weekly'])` kodu ile model veri setine uydurulur. Model, `cltv_df` DataFrame'indeki `frequency`, `recency_cltv_weekly` ve `T_weekly` sütunlarını kullanarak müşteri davranışlarını tanımlayan parametrelerini öğrenir.

BG/NBD modelinin başarılı bir şekilde kurulması, müşterilerin gelecekteki satın alma davranışlarına ilişkin tahminler yapmamıza olanak tanır ve CLTV hesaplamasının bir parçasını oluşturur.

---

In [ ]:
###############################################################
# 1.a. 3 aylık satın alma tahmini
###############################################################

# BG/NBD modeli (bgf) kullanarak sonraki 3 ay (12 hafta) için beklenen satın alma sayısını tahmin et
# predict fonksiyonu, modelin öğrendiği parametreleri ve müşterinin recency, frequency, T_weekly değerlerini kullanarak tahmini hesaplar.
cltv_df["exp_sales_3_month"] = bgf.predict(4*3, # 4 hafta/ay * 3 ay = 12 hafta
                                           cltv_df['frequency'],
                                           cltv_df['recency_cltv_weekly'],
                                           cltv_df['T_weekly'])

###############################################################
# 1.b. 6 aylık satın alma tahmini
###############################################################

# BG/NBD modeli (bgf) kullanarak sonraki 6 ay (24 hafta) için beklenen satın alma sayısını tahmin et
cltv_df["exp_sales_6_month"] = bgf.predict(4*6, # 4 hafta/ay * 6 ay = 24 hafta
                                           cltv_df['frequency'],
                                           cltv_df['recency_cltv_weekly'],
                                           cltv_df['T_weekly'])

# Tahmin sütunlarının başarıyla eklendiğini belirten bir mesaj yazdır
print("Expected sales predictions added (3 and 6 months).")

Expected sales predictions added (3 and 6 months).


* * *

**Gelecekteki Satın Alma Sayısının Tahmin Edilmesi (BG/NBD Modeli)**

BG/NBD modeli kurulduktan sonra, bu modeli kullanarak her bir müşterinin gelecekte belirli bir zaman diliminde yapması beklenen satın alma sayısını tahmin edebiliriz. Bu adımda, müşterilerin sonraki 3 ay (12 hafta) ve 6 ay (24 hafta) içinde kaç adet satın alma yapması beklendiğini tahmin ediyoruz.

*   **Kod Açıklaması:**

    *   `bgf.predict(t, frequency, recency, T)`: BG/NBD modelinin tahmin fonksiyonudur.
        *   `t`: Tahmin yapılmak istenen gelecekteki zaman dilimi (hafta cinsinden).
        *   `frequency`: Her müşterinin geçmişteki toplam sipariş sayısı.
        *   `recency`: Her müşterinin ilk ve son siparişi arasındaki süre (hafta cinsinden).
        *   `T`: Her müşterinin ilk siparişi ile analiz tarihi arasındaki toplam süre (hafta cinsinden).

    *   `cltv_df["exp_sales_3_month"] = bgf.predict(4*3, ...)`: Bu satır, her müşteri için sonraki 3 ay (4 hafta/ay * 3 ay = 12 hafta) içindeki beklenen satın alma sayısını tahmin eder ve bu tahminleri `cltv_df` DataFrame'ine "exp_sales_3_month" adında yeni bir sütun olarak ekler.
    *   `cltv_df["exp_sales_6_month"] = bgf.predict(4*6, ...)`: Benzer şekilde, bu satır her müşteri için sonraki 6 ay (4 hafta/ay * 6 ay = 24 hafta) içindeki beklenen satın alma sayısını tahmin eder ve "exp_sales_6_month" sütununa ekler.

Bu tahminler, CLTV hesaplamasının bir parçası olarak müşterinin gelecekteki toplam değerini tahmin etmek için kullanılacaktır.

---

In [ ]:
###############################################################
# 2. Gamma-Gamma Modeli
###############################################################

# Gamma-Gamma modelini başlat
# penalizer_coef, aşırı uyumu (overfitting) önlemek için kullanılan bir düzenlileştirme parametresidir.
ggf = GammaGammaFitter(penalizer_coef=0.001) # Using 0.001 as in BG/NBD for consistency, or 0.01 as in original code? Let's stick to 0.01 as in the user's provided code.
ggf = GammaGammaFitter(penalizer_coef=0.01)

# Gamma-Gamma modelini veri setine uygula (fit et)
# Model, müşterilerin sıklığı (frequency) ve ortalama işlem değeri (monetary_cltv_avg) verilerini kullanarak parametrelerini öğrenir.
# monetary_value: Ortalama işlem değeri sütunu
# frequency: Sıklık sütunu
ggf.fit(cltv_df['frequency'],
        cltv_df['monetary_cltv_avg'])

# Gamma-Gamma modelini kullanarak her müşteri için beklenen ortalama işlem değerini tahmin et
# conditional_expected_average_profit fonksiyonu, modelin öğrendiği parametreleri ve müşterinin frequency, monetary_cltv_avg değerlerini kullanarak tahmini hesaplar.
cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                       cltv_df['monetary_cltv_avg'])

# Model uydurma ve ortalama değer tahmininin tamamlandığını belirten mesaj
print("\nGamma-Gamma model fitted and average values predicted!")

###############################################################
# 3. 6 aylık CLTV hesaplama
###############################################################

# BG/NBD ve Gamma-Gamma modellerini kullanarak 6 aylık müşteri yaşam boyu değerini (CLTV) hesapla
# customer_lifetime_value fonksiyonu, iki modelin çıktılarını birleştirerek nihai CLTV'yi tahmin eder.
cltv = ggf.customer_lifetime_value(bgf, # BG/NBD modeli nesnesi
                                   cltv_df['frequency'], # Sıklık sütunu
                                   cltv_df['recency_cltv_weekly'], # Recency sütunu (haftalık)
                                   cltv_df['T_weekly'], # T sütunu (haftalık)
                                   cltv_df['monetary_cltv_avg'], # Ortalama işlem değeri sütunu
                                   time=6,  # CLTV'nin hesaplanacağı zaman dilimi (ay cinsinden)
                                   freq="W",  # Zaman birimi (Weekly - Haftalık)
                                   discount_rate=0.01) # Gelecekteki nakit akışlarının iskonto oranı

# Hesaplanan CLTV değerlerini cltv_df DataFrame'ine yeni bir sütun olarak ekle
cltv_df["cltv"] = cltv

# CLTV hesaplama işleminin tamamlandığını belirten mesaj
print("\n6-month CLTV calculated and added!")

# Hesaplanan CLTV değerlerine göre müşterileri sırala ve en yüksek CLTV'ye sahip ilk 20 müşteriyi görüntüle
print("\nTop 20 customers by CLTV:")
print(cltv_df.sort_values("cltv", ascending=False).head(20), "\n")


Gamma-Gamma model fitted and average values predicted!

6-month CLTV calculated and added!

Top 20 customers by CLTV:
                                customer_id  recency_cltv_weekly  T_weekly  \
9055   47a642fe-975b-11eb-8c2a-000d3a38a36f                 2.86      7.86   
13880  7137a5c0-7aad-11ea-8f20-000d3a38a36f                 6.14     13.14   
17323  f59053e2-a503-11e9-a2fc-000d3a38a36f                51.71    101.00   
12438  625f40a2-5bd2-11ea-98b0-000d3a38a36f                74.29     74.57   
7330   a4d534a2-5b1b-11eb-8dbd-000d3a38a36f                62.71     67.29   
8868   9ce6e520-89b0-11ea-a6e7-000d3a38a36f                 3.43     34.43   
6402   851de3b4-8f0c-11eb-8cb8-000d3a38a36f                 8.29      9.43   
6666   53fe00d4-7b7a-11eb-960b-000d3a38a36f                 9.71     13.00   
19538  55d54d9e-8ac7-11ea-8ec0-000d3a38a36f                52.57     58.71   
14858  031b2954-6d28-11eb-99c4-000d3a38a36f                14.86     15.57   
17963  8fd88976-6708-11

* * *

**Gamma-Gamma Modeli ve CLTV Hesaplama**

Bu adımda, müşteri yaşam boyu değerini (CLTV) tahmin etmenin son aşamaları gerçekleştirilmiştir: Gamma-Gamma modelinin kurulması, ortalama işlem değerlerinin tahmin edilmesi ve nihai 6 aylık CLTV değerlerinin hesaplanması.

1.  **Gamma-Gamma Modelinin Kurulması (`ggf.fit`)**:
    *   Gamma-Gamma modeli, müşterinin gelecekteki her bir işlemi başına düşen ortalama kar veya değeri tahmin etmek için kullanılır.
    *   Model, `cltv_df` DataFrame'indeki `frequency` (alışveriş sıklığı) ve `monetary_cltv_avg` (ortalama işlem değeri) sütunlarını kullanarak kurulmuştur.
    *   `penalizer_coef=0.01` parametresi, modelin aşırı uyumunu önlemek için kullanılmıştır.

2.  **Beklenen Ortalama İşlem Değerinin Tahmin Edilmesi (`ggf.conditional_expected_average_profit`)**:
    *   Kurulan Gamma-Gamma modeli kullanılarak, her bir müşteri için gelecekteki işlemlerinden elde edilmesi beklenen ortalama değer (`exp_average_value`) tahmin edilmiştir. Bu tahmin, müşterinin geçmişteki sıklığı ve ortalama harcamasına dayanır.

3.  **6 Aylık CLTV'nin Hesaplanması (`ggf.customer_lifetime_value`)**:
    *   Nihai Müşteri Yaşam Boyu Değeri (CLTV), BG/NBD modelinden elde edilen beklenen satın alma sayısı ile Gamma-Gamma modelinden elde edilen beklenen ortalama işlem değerinin birleştirilmesiyle hesaplanır.
    *   `customer_lifetime_value` fonksiyonu, BG/NBD modeli (`bgf`), `cltv_df` içindeki `frequency`, `recency_cltv_weekly`, `T_weekly`, `monetary_cltv_avg` metrikleri, tahmin süresi (`time=6` ay) ve zaman birimi (`freq="W"` hafta) gibi parametreleri alarak 6 aylık CLTV değerlerini hesaplar.
    *   `discount_rate=0.01` gelecekteki nakit akışlarını iskonto etmek için kullanılan orandır.
    *   Hesaplanan CLTV değerleri, `cltv_df` DataFrame'ine "cltv" adında yeni bir sütun olarak eklenmiştir.

Son olarak, en yüksek CLTV değerine sahip müşterileri görmek için DataFrame "cltv" sütununa göre azalan şekilde sıralanmış ve ilk 20 müşteri görüntülenmiştir. Bu müşteriler, işletme için en değerli müşteriler olarak değerlendirilebilir ve özel pazarlama stratejileri için hedef grup olabilirler.

---

---

**GÖREV 4: CLTV'ye GÖRE SEGMENTLERİN OLUŞTURULMASI**

Bu bölüm, hesaplanan müşteri yaşam boyu değeri (CLTV) metriklerine dayanarak müşterileri farklı segmentlere ayırır.

In [ ]:
###############################################################
# 1. Segmentleme (4 gruba ayırma)
###############################################################

# cltv sütununu çeyrek (quartile) bazında 4 eşit gruba ayırır.
# labels=["D", "C", "B", "A"] ile segmentlere D (en düşük CLTV) den A (en yüksek CLTV) ye doğru etiket atar.
cltv_df["cltv_segment"] = pd.qcut(cltv_df["cltv"], 4, labels=["D", "C", "B", "A"])
print("\nCustomers segmented into 4 groups (A-D).")

# Oluşturulan segmentlerin dağılımını (her segmentteki müşteri sayısı) göster
print("\nSegment distribution:")
print(cltv_df["cltv_segment"].value_counts())

# Segmentlenmiş DataFrame'in ilk birkaç satırını görüntüle
print("\nSample of segmented customers:\n")
print(cltv_df.head())


Customers segmented into 4 groups (A-D).

Segment distribution:
cltv_segment
D    4987
C    4986
B    4986
A    4986
Name: count, dtype: int64

Sample of segmented customers:

                            customer_id  recency_cltv_weekly  T_weekly  \
0  cc294636-19f0-11eb-8d74-000d3a38a36f                17.00     30.57   
1  f431bd5a-ab7b-11e9-a2fc-000d3a38a36f               209.86    224.86   
2  69b69676-1a40-11ea-941b-000d3a38a36f                52.29     78.86   
3  1854e56c-491f-11eb-806e-000d3a38a36f                 1.57     20.86   
4  d6ea1074-f1f5-11e9-9346-000d3a38a36f                83.14     95.43   

   frequency  monetary_cltv_avg  exp_sales_3_month  exp_sales_6_month  \
0          5             187.87               0.97               1.95   
1         21              95.88               0.98               1.97   
2          5             117.06               0.67               1.34   
3          2              60.98               0.70               1.40   
4          2 

* * *

**CLTV'ye Göre Segmentlerin Oluşturulması**

Bu adımda, hesaplanan 6 aylık Müşteri Yaşam Boyu Değeri (CLTV) metrikleri kullanılarak müşteriler farklı segmentlere ayrılmıştır. Bu segmentasyon, pazarlama stratejilerinin daha etkili bir şekilde hedeflenmesine olanak tanır.

*   **Segmentasyon Yöntemi:** Müşteriler, `cltv` değerlerine göre çeyrek (quartile) bazında 4 eşit gruba ayrılmıştır.
    *   `pd.qcut(cltv_df["cltv"], 4, labels=["D", "C", "B", "A"])` kodu, `cltv` sütununu alır ve veriyi dört eşit parçaya böler.
    *   `labels=["D", "C", "B", "A"]` parametresi, en düşük CLTV değerine sahip çeyreği "D" segmenti, bir sonraki çeyreği "C", sonraki çeyreği "B" ve en yüksek CLTV değerine sahip çeyreği "A" segmenti olarak etiketler. Bu etiketleme, "A" segmentinin en değerli müşterileri temsil etmesini sağlar.

*   **Sonuç:** Oluşturulan segmentler, `cltv_df` DataFrame'ine "cltv_segment" adında yeni bir sütun olarak eklenmiştir. Segmentlerin dağılımına bakıldığında, her segmentin yaklaşık olarak eşit sayıda müşteri içerdiği görülmektedir (çeyrek bazında ayırma yapıldığı için beklenildiği gibi).

Bu segmentasyon, farklı müşteri gruplarının ihtiyaçlarını ve değerlerini anlamak için bir temel oluşturur ve bu gruplara yönelik özelleştirilmiş pazarlama ve iletişim stratejileri geliştirmeye yardımcı olur.

---

---

**BONUS: TÜM SÜRECİ FONKSİYONLAŞTIRMA**

Bu bölüm, projenin tüm adımlarını (veri hazırlığı, modelleme, CLTV hesaplama ve segmentasyon) tek bir fonksiyon altında birleştirerek sürecin yeniden kullanılabilir ve daha kolay yönetilebilir hale gelmesini sağlar.

In [ ]:
# Tüm CLTV Sürecini Fonksiyonlaştırma
def create_cltv_df(dataframe):
    # 1. Aykırı değerleri baskılama
    columns = ["order_num_total_ever_online", "order_num_total_ever_offline",
               "customer_value_total_ever_offline", "customer_value_total_ever_online"]
    for col in columns:
        replace_with_thresholds(dataframe, col)

    # 2. Yeni değişkenler oluşturma (Toplam sipariş ve toplam harcama)
    dataframe["order_num_total"] = dataframe["order_num_total_ever_online"] + dataframe["order_num_total_ever_offline"]
    dataframe["customer_value_total"] = dataframe["customer_value_total_ever_offline"] + dataframe["customer_value_total_ever_online"]

    # 3. Tarih sütunlarını datetime formatına dönüştürme
    date_columns = dataframe.columns[dataframe.columns.str.contains("date")]
    dataframe[date_columns] = dataframe[date_columns].apply(pd.to_datetime)

    # 4. Analiz tarihi belirleme
    analysis_date = dt.datetime(2021, 6, 1)

    # 5. CLTV veri yapısı oluşturma (recency, T, frequency, monetary)
    cltv_df = pd.DataFrame()
    cltv_df["customer_id"] = dataframe["master_id"]
    cltv_df["recency_cltv_weekly"] = ((dataframe["last_order_date"] - dataframe["first_order_date"]).dt.days) / 7
    cltv_df["T_weekly"] = ((analysis_date - dataframe["first_order_date"]).dt.days) / 7
    cltv_df["frequency"] = dataframe["order_num_total"]
    cltv_df["monetary_cltv_avg"] = dataframe["customer_value_total"] / dataframe["order_num_total"]

    # Frequency değeri 1'den büyük olan müşterileri filtrele (Model gereksinimi)
    cltv_df = cltv_df[(cltv_df['frequency'] > 1)]

    # Frequency sütununu integer'a dönüştür (Model gereksinimi)
    cltv_df['frequency'] = cltv_df['frequency'].astype(int)


    # 6. BG-NBD Modeli kurma ve tahmin
    bgf = BetaGeoFitter(penalizer_coef=0.001)
    bgf.fit(cltv_df['frequency'],
            cltv_df['recency_cltv_weekly'],
            cltv_df['T_weekly'])
    cltv_df["exp_sales_3_month"] = bgf.predict(4*3,
                                               cltv_df['frequency'],
                                               cltv_df['recency_cltv_weekly'],
                                               cltv_df['T_weekly'])
    cltv_df["exp_sales_6_month"] = bgf.predict(4*6,
                                               cltv_df['frequency'],
                                               cltv_df['recency_cltv_weekly'],
                                               cltv_df['T_weekly'])

    # 7. Gamma-Gamma Modeli kurma ve tahmin
    ggf = GammaGammaFitter(penalizer_coef=0.01)
    # Monetary değerlerinin pozitif olduğunu kontrol etmeden fit ediliyor (varsayım kontrolü yapılabilir)
    ggf.fit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])
    cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                           cltv_df['monetary_cltv_avg'])

    # 8. CLTV Hesaplama (6 aylık)
    cltv = ggf.customer_lifetime_value(bgf,
                                       cltv_df['frequency'],
                                       cltv_df['recency_cltv_weekly'],
                                       cltv_df['T_weekly'],
                                       cltv_df['monetary_cltv_avg'],
                                       time=6,
                                       freq="W",
                                       discount_rate=0.01)
    cltv_df["cltv"] = cltv

    # 9. Segmentleme (4 gruba ayırma)
    cltv_df["cltv_segment"] = pd.qcut(cltv_df["cltv"], 4, labels=["D", "C", "B", "A"])

    return cltv_df

# Fonksiyonu çağırma ve sonucu bir değişkene atama
cltv_result = create_cltv_df(df)
print("\nCLTV process completed successfully!\n")
print(cltv_result.head())

/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)



CLTV process completed successfully!

                            customer_id  recency_cltv_weekly  T_weekly  \
0  cc294636-19f0-11eb-8d74-000d3a38a36f                17.00     30.57   
1  f431bd5a-ab7b-11e9-a2fc-000d3a38a36f               209.86    224.86   
2  69b69676-1a40-11ea-941b-000d3a38a36f                52.29     78.86   
3  1854e56c-491f-11eb-806e-000d3a38a36f                 1.57     20.86   
4  d6ea1074-f1f5-11e9-9346-000d3a38a36f                83.14     95.43   

   frequency  monetary_cltv_avg  exp_sales_3_month  exp_sales_6_month  \
0          5             187.87               0.97               1.95   
1         21              95.88               0.98               1.97   
2          5             117.06               0.67               1.34   
3          2              60.98               0.70               1.40   
4          2             104.99               0.40               0.79   

   exp_average_value   cltv cltv_segment  
0             193.63 395.73       

* * *

**BONUS: Tüm CLTV Sürecinin Fonksiyonlaştırılması**

Bu adımda, daha önceki görevlerde adım adım gerçekleştirdiğimiz tüm CLTV hesaplama ve segmentasyon sürecini tek bir Python fonksiyonu (`create_cltv_df`) içine alınmıştır. Bu fonksiyon, veri setini girdi olarak alır ve işlenmiş, CLTV değerleri hesaplanmış ve segmentlere ayrılmış bir DataFrame döndürür.

Fonksiyonun temel adımları şunlardır:

1.  Aykırı değerlerin belirlenen eşiklerle değiştirilmesi (`replace_with_thresholds` fonksiyonu kullanılarak).
2.  Toplam sipariş sayısı (`order_num_total`) ve toplam harcama değeri (`customer_value_total`) gibi yeni değişkenlerin oluşturulması.
3.  Tarih sütunlarının datetime formatına dönüştürülmesi.
4.  CLTV hesaplamaları için gerekli temel metrikleri (`recency_cltv_weekly`, `T_weekly`, `frequency`, `monetary_cltv_avg`) içeren `cltv_df` DataFrame'inin oluşturulması.
5.  Modelleme gereksinimlerine uyması için `cltv_df` DataFrame'inin filtrelenmesi (frequency > 0 olan müşteriler gibi) ve `frequency` sütununun integer tipine dönüştürülmesi.
6.  BG/NBD modelinin uydurulması ve gelecekteki satın alma sayılarının tahmin edilmesi (3 ve 6 aylık).
7.  Gamma-Gamma modelinin uydurulması ve beklenen ortalama işlem değerlerinin tahmin edilmesi.
8.  BG/NBD ve Gamma-Gamma modellerinin çıktıları kullanılarak nihai 6 aylık CLTV değerlerinin hesaplanması.
9.  Hesaplanan CLTV değerlerine göre müşterilerin segmentlere ayrılması (4 farklı gruba).

Bu fonksiyonlaştırma, CLTV analizini farklı veri setleri üzerinde veya farklı zamanlarda kolayca tekrarlanabilir hale getirir, kodun düzenliliğini ve yeniden kullanılabilirliğini artırır. Kod bloğunun son kısmında, tanımlanan bu fonksiyon, orijinal `df` DataFrame'i üzerinde çağrılarak sonuç `cltv_result` değişkenine atanmış ve ilk birkaç satırı yazdırılmıştır.

* * *

* * *

# Projenin Sonuçları, Faydaları ve Kullanılan Beceriler

Bu proje kapsamında, FLO müşterilerinin geçmiş alışveriş davranışları analiz edilerek Müşteri Yaşam Boyu Değeri (CLTV) tahmin edildi ve müşteriler bu değere göre segmentlere ayrıldı.

**Projenin Sonuçları:**

*   Müşterilerin Recency, Frequency ve Monetary metrikleri hesaplandı.
*   BG/NBD modeli kullanılarak müşterilerin gelecekteki satın alma sayıları tahmin edildi.
*   Gamma-Gamma modeli kullanılarak müşterilerin ortalama işlem değerleri tahmin edildi.
*   Bu iki modelin çıktıları birleştirilerek her müşterinin 6 aylık CLTV değeri hesaplandı.
*   CLTV değerlerine göre müşteriler 4 farklı segmente (A, B, C, D) ayrıldı.

---
**Projenin Faydaları:**

*   **Hedeflenmiş Pazarlama:** CLTV segmentasyonu sayesinde, farklı müşteri gruplarına yönelik özelleştirilmiş pazarlama kampanyaları oluşturulabilir. Örneğin, yüksek CLTV'ye sahip "A" segmentindeki müşterilere sadakat programları veya özel teklifler sunularak bu müşterilerin elde tutulması sağlanabilir. Düşük CLTV'ye sahip "D" segmentindeki müşterilere ise yeniden aktivasyon kampanyaları düzenlenebilir.
*   **Kaynak Optimizasyonu:** Pazarlama bütçesi ve diğer kaynaklar, en değerli veya en potansiyelli segmentlere odaklanarak daha verimli kullanılabilir.
*   **Müşteri İlişkileri Yönetimi (CRM):** Müşterilerin yaşam boyu değerini anlamak, daha güçlü müşteri ilişkileri kurmaya ve müşteri memnuniyetini artırmaya yardımcı olur.
*   **İş Kararları:** CLTV tahminleri, ürün geliştirme, fiyatlandırma stratejileri ve müşteri edinme maliyetleri gibi konularda daha bilinçli iş kararları alınmasını destekler.

---
**Kullanılan Beceriler ve Teknolojiler:**

*   **Python Programlama Dili:** Veri işleme, analiz ve modelleme adımları için kullanılmıştır.
*   **Pandas Kütüphanesi:** Veri manipülasyonu ve DataFrame işlemleri için kullanılmıştır.
*   **Datetime Kütüphanesi:** Tarih ve saat verileriyle çalışmak için kullanılmıştır.
*   **Lifetimes Kütüphanesi:** BG/NBD ve Gamma-Gamma modelleri ile CLTV tahmini için kullanılmıştır.
*   **Scikit-learn Kütüphanesi (MinMaxScaler):** Sayısal değişkenleri ölçeklendirme (bu projede kullanılmadı ancak potansiyel bir adımdır) gibi makine öğrenimi hazırlık adımları için.
*   **Veri Temizleme ve Ön İşleme:** Aykırı değerlerin tespiti ve baskılanması gibi veri hazırlık teknikleri uygulanmıştır.
*   **Özellik Mühendisliği:** Mevcut değişkenlerden yeni anlamlı değişkenler (toplam sipariş, toplam harcama) oluşturulmuştur.
*   **İstatistiksel Modelleme:** BG/NBD ve Gamma-Gamma gibi istatistiksel modeller kullanılarak müşteri davranışları tahmin edilmiştir.
*   **Müşteri Segmentasyonu:** CLTV değerlerine dayanarak müşteriler anlamlı gruplara ayrılmıştır.
*   **Veri Analizi ve Yorumlama:** Hesaplanan metrikler ve oluşturulan segmentler yorumlanarak işe yarar içgörüler elde edilmiştir.
*   **Fonksiyonel Programlama:** Kodun tekrar kullanılabilirliğini artırmak için tüm sürecin bir fonksiyon içine alınması.

Bu proje, veri odaklı pazarlama stratejileri geliştirmek ve müşteri değerini maksimize etmek için CLTV analizinin nasıl kullanılacağına dair kapsamlı bir örnek sunmaktadır.